In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import scipy.integrate as itp
from tqdm.notebook import tqdm

plt.rc('font', size=18)
plt.rc('axes', titlesize=18)
plt.rc('axes', labelsize=18)
plt.rc('xtick', labelsize=18)
plt.rc('ytick', labelsize=18)
plt.rc('legend', fontsize=10)
plt.rc('mathtext', fontset='cm')
plt.rc('text', usetex=True)
#plt.rc('font', family='nimbus sans narrow') 

In [ ]:
from dabry.flowfield import GyreFF

In [ ]:
gyre = GyreFF(0.5, 0, 1, 1, 1)
def v_f(x, t):
    return gyre.value(t, x)

def grad_v_f(x, t):
    return gyre.d_value(t, x)
    
def F(y, t):
    state, adjoint = y[:2], y[2:]
    return np.concatenate((-v_r * adjoint / np.linalg.norm(adjoint) + v_f(state, t),
                    -grad_v_f(state, t).transpose() @ adjoint))
def F_obs(y, t):
    state, adjoint = y[:2], y[2:]
    return np.concatenate((-v_r * adjoint / np.linalg.norm(adjoint) + v_f(state, t),
                    -grad_v_f(state, t).transpose() @ adjoint))

In [ ]:
#nx, ny = 101, 101
nx, ny = 31, 31
bounds = np.array(((-0.6, 0.6), (-0.5, 0.5)))
xx, yy = np.meshgrid(np.linspace(*bounds[0], nx), np.linspace(*bounds[1], ny), indexing='ij')
ttt, xxx, yyy = np.meshgrid(np.linspace(0, 0, 1), np.linspace(*bounds[0], nx), np.linspace(*bounds[1], ny), indexing='ij')
grid = np.stack((np.squeeze(ttt), np.squeeze(xxx), np.squeeze(yyy)), axis=-1)
wind_values = gyre.value_vec(grid)

In [ ]:
wind_grads = gyre.d_value_vec(grid)

In [ ]:
np.linalg.norm(wind_grads, axis=(-2, -1)).max()

In [ ]:
wind_values.shape

In [ ]:
def wind_plot(no_wind_norm=False):    
    fig, ax = plt.subplots()
    
    if not no_wind_norm:
        ax.pcolormesh(xx, yy, np.linalg.norm(wind_values, axis=-1), cmap='turbo', shading='gouraud')
    rect = patches.Rectangle((bounds[0, 0], bounds[1, 0]), bounds[0, 1] - bounds[0, 0], bounds[1, 1] - bounds[1, 0], 
                             facecolor='white', alpha=0.4)
    ax.add_patch(rect)
    n_quiv = 2
    off = 1
    ax.grid(True, color='black', alpha=0.5)
    ax.quiver(xx[off::n_quiv, off::n_quiv], yy[off::n_quiv, off::n_quiv], 
              wind_values[off::n_quiv, off::n_quiv, 0], wind_values[off::n_quiv, off::n_quiv, 1], alpha=0.6)
    
    ax.scatter(*np.array((0, 0)), color='black', edgecolor='white', s=100, zorder=10)
    #ax.scatter(*np.array((1, 0)), color='black', edgecolor='white', s=200, marker='*', zorder=10)
    return fig, ax

In [ ]:
fig, ax = wind_plot()

n_traj, n_t = 301, 60
v_r, a = 1., 1.
trajs = np.zeros((n_traj, n_t, 4))
x0 = np.zeros(2)
angles = np.linspace(0, 2*np.pi, n_traj)

T = 0.35
for i, angle in enumerate(angles):
    y0 = np.array((0., 0., -np.cos(angle), -np.sin(angle)))
    trajs[i, :] = itp.odeint(F, y0, np.linspace(0, T, n_t))

for i in range(0, trajs.shape[0], 3):
    #ax.plot(trajs[i, :, 0], trajs[i, :, 1], linewidth=2.5, color='white')
    ax.plot(trajs[i, :, 0], trajs[i, :, 1], linewidth=1, color='black')
    ax.scatter(trajs[i, -1, 0], trajs[i, -1, 1], color='black', edgecolor='white', marker='D', zorder=10)

ax.plot(trajs[:, -1, 0], trajs[:, -1, 1], linewidth=2, color='blue')

# ax.set_xlim(-0.5, 0.5)
# ax.set_ylim(-0.5, 0.5)
#ax.set_xticks([0, 0.2, 0.4, 0.6, 0.8 ,1])
#ax.set_yticks(np.arange(-0.4, 0.41, 0.2))
ax.set_title(r'\textbf{Gyre flow field}')
#plt.savefig('gyre_extremals/gyre_extremals.png', dpi=600)
#plt.savefig('gyre_extremals/gyre_extremals.pdf')
plt.show()

In [ ]:
#nx, ny = 101, 101
nx, ny = 51, 51
bounds = np.array(((-1.1, 1.1), (-0.8, 0.8)))
xx, yy = np.meshgrid(np.linspace(*bounds[0], nx), np.linspace(*bounds[1], ny), indexing='ij')
ttt, xxx, yyy = np.meshgrid(np.linspace(0, 0, 1), np.linspace(*bounds[0], nx), np.linspace(*bounds[1], ny), indexing='ij')
grid = np.stack((np.squeeze(ttt), np.squeeze(xxx), np.squeeze(yyy)), axis=-1)
wind_values = gyre.value_vec(grid)

In [ ]:
def wind_plot(no_wind_norm=False):    
    fig, ax = plt.subplots()
    
    if not no_wind_norm:
        ax.pcolormesh(xx, yy, np.linalg.norm(wind_values, axis=-1), cmap='turbo', shading='gouraud')
    rect = patches.Rectangle((bounds[0, 0], bounds[1, 0]), bounds[0, 1] - bounds[0, 0], bounds[1, 1] - bounds[1, 0], 
                             facecolor='white', alpha=0.4)
    ax.add_patch(rect)
    n_quiv = 2
    off = 1
    ax.grid(True, color='black', alpha=0.5)
    ax.quiver(xx[off::n_quiv, off::n_quiv], yy[off::n_quiv, off::n_quiv], 
              wind_values[off::n_quiv, off::n_quiv, 0], wind_values[off::n_quiv, off::n_quiv, 1], alpha=0.6)
    
    ax.scatter(*np.array((0, 0)), color='black', edgecolor='white', s=100, zorder=10)
    #ax.scatter(*np.array((1, 0)), color='black', edgecolor='white', s=200, marker='*', zorder=10)
    return fig, ax

In [ ]:
n_traj, n_t = 3000, 60
v_r, a = 1., 1.
trajs = np.zeros((n_traj, n_t, 4))
x0 = np.zeros(2)
angles = np.linspace(0, 2*np.pi, n_traj)

T = 0.7
for i, angle in enumerate(angles):
    y0 = np.array((0., 0., -np.cos(angle), -np.sin(angle)))
    trajs[i, :] = itp.odeint(F, y0, np.linspace(0, T, n_t))

In [ ]:
fig, ax = wind_plot(no_wind_norm=True)

# for it in range(n_t - 1):
#     ax.plot(trajs[:, it, 0], trajs[:, it, 1], linewidth=2.5, alpha=0.2, color='green')

cmap = matplotlib.colormaps.get_cmap('hsv')


for i in range(trajs.shape[0]):
    ax.plot(trajs[i, :, 0], trajs[i, :, 1], color=cmap(i/trajs.shape[0]), alpha=0.7)

i1 = 105
i2 = 1395
i3 = 1605
i4 = 2895
ax.plot(trajs[:i1, -1, 0], trajs[:i1, -1, 1], linewidth=2, color='blue')
ax.plot(trajs[i1:i2, -1, 0], trajs[i1:i2, -1, 1], linewidth=2, color='red')
ax.plot(trajs[i2:i3, -1, 0], trajs[i2:i3, -1, 1], linewidth=2, color='blue')
ax.plot(trajs[i3:i4, -1, 0], trajs[i3:i4, -1, 1], linewidth=2, color='green')
ax.plot(trajs[i4:, -1, 0], trajs[i4:, -1, 1], linewidth=2, color='blue')

# for i in [2, 6, 10]:
#     ax.plot(trajs[i, :, 0], trajs[i, :, 1], linewidth=2.5, color='white')
#     ax.plot(trajs[i, :, 0], trajs[i, :, 1], linewidth=1.5, color='black' if i != 4 else 'red')
#     ax.scatter(trajs[i, -1, 0], trajs[i, -1, 1], color='black', edgecolor='white', marker='D', zorder=10)

# ax.annotate(r'$\mathcal{E}_{\mathbf{x}_0}^{\mathbf{x}}(t)$', (0.4, -0.22), xytext=(0.5, -0.3), color='blue',
#             bbox=dict(facecolor='white', edgecolor='blue', boxstyle='round,pad=0.2', linewidth=0.5),
#            arrowprops=dict(edgecolor='blue', arrowstyle='-', linewidth=0.5))

# ax.annotate(r'$\mathcal{S}_{\mathbf{x}_0}^{\mathbf{x}}(t)$', (0, -0.2), color='green',
#             bbox=dict(facecolor='white', edgecolor='green', boxstyle='round,pad=0.2', linewidth=0.5))

ax.set_xlim(-1.1, 1.1)
#ax.set_xticks([0, 0.2, 0.4, 0.6, 0.8 ,1])
#ax.set_yticks(np.arange(-0.4, 0.41, 0.2))
#ax.set_title(f'Linear flow field', fontweight="bold")
#plt.savefig('linear_extremals/linear_extremals_set.png', dpi=600)
plt.show()

In [ ]:
fig, ax = plt.subplots()
for spine_type in ['top', 'right', 'bottom', 'left']:
    ax.spines[spine_type].set_visible(False)
        
ax.tick_params(axis='both', which='both', length=0)
ax.set_xticklabels([])
ax.set_yticklabels([])

ax.scatter(*np.array((0, 0)), color='black', edgecolor='white', s=100, zorder=10)

n_traj, n_t = 1000, 60
v_r, a = 1., 1.
trajs = np.zeros((n_traj, n_t, 4))
x0 = np.zeros(2)
angles = np.linspace(0, 2*np.pi, n_traj)

T = 0.6
for i, angle in enumerate(angles):
    y0 = np.array((0., 0., -np.cos(angle), -np.sin(angle)))
    trajs[i, :] = itp.odeint(F, y0, np.linspace(0, T, n_t))
    
ax.plot(trajs[:, -1, 0], trajs[:, -1, 1], linewidth=2, color='blue', alpha=0.5)

step = 2
for i in range(0, n_traj, step):
    ax.plot(trajs[i, :, 0], trajs[i, :, 1], linewidth=2.5, color='white')
    ax.plot(trajs[i, :, 0], trajs[i, :, 1], linewidth=1.5, color='black')
    ax.scatter(trajs[i, -1, 0], trajs[i, -1, 1], color='black', edgecolor='white', marker='D', zorder=10)
    if i < n_traj - 1:
        ax.plot([trajs[i, -1, 0], trajs[i + step, -1, 0]], 
                [trajs[i, -1, 1], trajs[i + step, -1, 1]], linewidth=2, color='red', alpha=0.8)
        step_it = 10
        for it in range(0, n_t, step_it):
            coords = np.stack((trajs[i, it, :2], trajs[i + step, it, :2], trajs[i + step, min(it + step_it, n_t - 1), :2]))
            tri = patches.Polygon(coords, color='orange', alpha=0.7)
            ax.add_patch(tri)
            coords = np.stack((trajs[i, it, :2], trajs[i, min(it + step_it, n_t - 1), :2], trajs[i + step, min(it + step_it, n_t - 1), :2]))
            tri = patches.Polygon(coords, color='green', alpha=0.7)
            ax.add_patch(tri)

ax.set_xlim(-0.6, 0.6)
ax.set_xticks(np.arange(-0.6, 0.61, 0.2))
ax.set_yticks(np.arange(-0.4, 0.41, 0.2))
ax.axis('equal')
#ax.set_title(f'Linear flow field', fontweight="bold")
#plt.savefig('advection_extremals/advection_extremals_set_approx.png', dpi=600)
plt.show()

In [ ]:
v_a = 1

# Kularatne 2016

x_init = np.array((0.6, 0.6))
x_target = np.array((2.4, 2.4))
bl = np.array((0.4, 0.4))
tr = np.array((2.6, 2.6))

gyre = GyreFF(0.5, 0.5, 2, 2, 1)

In [ ]:
#nx, ny = 101, 101
nx, ny = 51, 51
bounds = np.array((bl, tr)).transpose()
xx, yy = np.meshgrid(np.linspace(*bounds[0], nx), np.linspace(*bounds[1], ny), indexing='ij')
ttt, xxx, yyy = np.meshgrid(np.linspace(0, 0, 1), np.linspace(*bounds[0], nx), np.linspace(*bounds[1], ny), indexing='ij')
grid = np.stack((np.squeeze(ttt), np.squeeze(xxx), np.squeeze(yyy)), axis=-1)
wind_values = gyre.value_vec(grid)

In [ ]:
d_values = gyre.d_value_vec(grid)

In [ ]:
np.max(d_values)

In [ ]:
L = np.sqrt(2) * np.max(np.abs(d_values))
print(L)

In [ ]:
L_mean = np.sqrt(2) * np.mean(np.abs(d_values))
print(L_mean)

In [ ]:
n_L = int(np.ceil(2 * np.pi / 0.2 * np.exp(L * T)))
print('n_L: ', n_L)
n_L_mean = int(np.ceil(2 * np.pi / 0.2 * np.exp(L_mean * T)))
print('n_L_mean: ', n_L_mean)

In [ ]:
n_traj, n_t = 10000, 60
v_r, a = 1., 1.
trajs = np.zeros((n_traj, n_t, 4))
x0 = x_init
angles = np.linspace(0, 2*np.pi, n_traj)

T = 1.6
for i, angle in enumerate(tqdm(angles)):
    y0 = np.hstack((x_init, (-np.cos(angle), -np.sin(angle))))
    trajs[i, :] = itp.odeint(F, y0, np.linspace(0, T, n_t))

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))


ax.pcolormesh(xx, yy, np.linalg.norm(wind_values, axis=-1), cmap='turbo', shading='gouraud')
rect = patches.Rectangle((bounds[0, 0], bounds[1, 0]), bounds[0, 1] - bounds[0, 0], bounds[1, 1] - bounds[1, 0], 
                         facecolor='white', alpha=0.4)
ax.add_patch(rect)
n_quiv = 2
off = 1
ax.grid(True, color='black', alpha=0.5)
ax.quiver(xx[off::n_quiv, off::n_quiv], yy[off::n_quiv, off::n_quiv], 
          wind_values[off::n_quiv, off::n_quiv, 0], wind_values[off::n_quiv, off::n_quiv, 1], alpha=0.6)

ax.plot(trajs[:, -1, 0], trajs[:, -1, 1], linewidth=2, color='blue')


for i in range(0, trajs.shape[0], 3):
    #ax.plot(trajs[i, :, 0], trajs[i, :, 1], linewidth=2.5, color='white')
    ax.plot(trajs[i, :, 0], trajs[i, :, 1], linewidth=1, color='black')
    ax.scatter(trajs[i, -1, 0], trajs[i, -1, 1], color='black', edgecolor='white', marker='D', zorder=10)

ax.scatter(*x_init, color='black', edgecolor='white', s=100, zorder=10)
ax.scatter(*x_target, color='black', edgecolor='white', s=200, marker='*', zorder=10)
circ = patches.Circle(x_target, 0.1, facecolor=(1, 0, 0, 0.5), edgecolor='red', linewidth=2)
ax.add_patch(circ)

ax.set_xticks(np.arange(0.5, 2.51, 0.5))
ax.set_yticks(np.arange(0.5, 2.51, 0.5))
ax.set_xlim(bl[0], tr[0])
ax.set_ylim(bl[1], tr[1])

plt.savefig('gyre_extremals/gyre_equisampling.pdf')